In [66]:

from training.generate import (generate_response, load_model_tokenizer_for_generate, 
                               get_special_token_id, get_special_token_id)
from training.consts import END_KEY, PROMPT_FORMAT, RESPONSE_KEY_NL
from training.trainer import PROMPT_FORMAT, create_data_set_from_json_list
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk


In [78]:
file_to_instruction_map = {
    "/opt/home/bo_ling/dataset/michelangelo_so_long.jsonl": "Please answer the following MA helpdesk questions:",
    "/opt/home/bo_ling/dataset/docstrans.jsonl": "Please finish the following doc translation tasks:",
    "/opt/home/bo_ling/dataset/eats_goldenset.jsonl": "Please answer Uber eats products relation questions:"
}
json_file = "/opt/home/bo_ling/dataset/eats_goldenset.jsonl"
dataset_file = json_file.replace(".jsonl", ".hf")
dataset = create_data_set_from_json_list(json_file,
                                         file_to_instruction_map=file_to_instruction_map) 
dataset = dataset.train_test_split(test_size=0.1)
#dataset.save_to_disk(dataset_file)

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-71d75ecb741b2dcb/0.0.0. Subsequent calls will reuse this data.


In [79]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/eats_goldenset.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 157792
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 17533
    })
})

In [71]:
data = []
max_question_length=500
max_answer_length=500
with open("/opt/home/bo_ling/dataset/doc_transcript_pii_data.json") as json_file:
    json_data = json.load(json_file)
for d in json_data:
    try:
        instruction = d['instruction']+":"
        input_text = d["input"][:max_question_length]
        output_text = "\n" + d["output"][:max_answer_length]
        text = PROMPT_FORMAT.format(instruction=instruction,input_text=input_text,output_text=output_text)
        data.append({
                "instruction": instruction,
                "input": input_text, 
                "output": output_text,
                "text": text
            })
    except:
        pass
def gen():
    for d in data:
        yield d
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.1)
#dataset.save_to_disk("/opt/home/bo_ling/dataset/doc_transcript_pii_data.hf")

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-2da5259a7e982007/0.0.0. Subsequent calls will reuse this data.


In [75]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/doc_transcript_pii_data.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 629797
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 69978
    })
})